# Cross LULC Zone change Matrix and NDVI Zone change Matrix

In [1]:
# Load the necessary Python packages for the analysis.
%matplotlib inline

import os
import datacube
import pickle
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# Change the default decimal precision used for printing dataframes.
pd.set_option("display.precision", 2)

In [3]:
# Set important parameters for the analysis.

# Directory where the results from previous notebooks are stored in.
output_dir = "results"

# Select an African country to carry out the analysis over.
country = "Tunisia"

# Select the initial and follow up year for change detection used in the previous notebooks.
initial_year = "2015"
followup_year = "2019"

In [4]:
# Custom dataframe styling functions.

def cross_matrix_set_background_color(df):

    # Define the background colors to be used for the input dataFrame cells.
    # based on the type of change.
    red = "background-color : red"
    purple = "background-color : #E5D1EF"
    orange = "background-color : orange"
    light_green = "background-color : #A8CD66"
    green = "background-color : green"

    # Create a copy of the input dataframe. 
    df1 = df.copy(deep=True)
    
    row_fill_value = df1.index[0][0]
    column_fill_value = df1.columns[0][0]
    
    # Define the color code for the types of changes.
    df1.loc[(row_fill_value, "Bad"), (column_fill_value, "Bad")] = red
    df1.loc[(row_fill_value, "Bad"), (column_fill_value, "Stable")] = orange
    df1.loc[(row_fill_value, "Bad"), (column_fill_value, "Excellent")] = light_green
    df1.loc[(row_fill_value, "Stable"), (column_fill_value, "Bad")] = orange
    df1.loc[(row_fill_value, "Stable"), (column_fill_value, "Stable")] = purple
    df1.loc[(row_fill_value, "Stable"), (column_fill_value, "Excellent")] = red
    df1.loc[(row_fill_value, "Excellent"), (column_fill_value, "Bad")] = light_green
    df1.loc[(row_fill_value, "Excellent"), (column_fill_value, "Stable")] = red
    df1.loc[(row_fill_value, "Excellent"), (column_fill_value, "Excellent")] = green

    return df1

def make_pretty(styler, title):
    # Add a title to the dataframe.
    styler.set_caption(title)
    # Increase font size and bold the dataframe title.
    title_style = dict(selector="caption", props=[("text-align", "right"), 
                                                  ("font-weight", "bold"), 
                                                  ("font-size", "14px")])
    # Rotate the level 0 row labels.
    row_labels_style = dict(selector="th.row_heading.level0", props=[("text-align", "left"),
                                                                     ("width", "100px")])
    
    column_labels_style = dict(selector="th.column_heading.level0", props=[("text-align", "left")])
    
    styler.set_table_styles([title_style, row_labels_style])
    return styler

In [5]:
# Create the reference Cross LULC Zone change and NDVI zone change matrix.

row_labels_level1 = ["Bad", "Stable", "Excellent"]
row_fill_value = "LULC Zone Change Matrix Result"
row_labels_level0 = np.full(shape=len(row_labels_level1), fill_value=row_fill_value)
row_labels_tuples = list(zip(row_labels_level0, row_labels_level1))
row_labels = pd.MultiIndex.from_tuples(row_labels_tuples)

column_labels_level1 = ["Bad", "Stable", "Excellent"]
column_fill_value = "NDVI Zone Change Matrix Result"
column_labels_level0 = np.full(shape=len(column_labels_level1), fill_value=column_fill_value)
column_labels_tuples = list(zip(column_labels_level0, column_labels_level1))
column_labels = pd.MultiIndex.from_tuples(column_labels_tuples)

# Create an empty dataframe.
ref_cross_matrix = pd.DataFrame(data=" ", index=row_labels, columns=column_labels)

# Define the types of changes.
ref_cross_matrix.loc[(row_fill_value, "Bad"), (column_fill_value, "Bad")] = "Mediocre"
ref_cross_matrix.loc[(row_fill_value, "Bad"), (column_fill_value, "Stable")] = "Bad"
ref_cross_matrix.loc[(row_fill_value, "Bad"), (column_fill_value, "Excellent")] = "Good"
ref_cross_matrix.loc[(row_fill_value, "Stable"), (column_fill_value, "Bad")] = "Bad"
ref_cross_matrix.loc[(row_fill_value, "Stable"), (column_fill_value, "Stable")] = "Stable"
ref_cross_matrix.loc[(row_fill_value, "Stable"), (column_fill_value, "Excellent")] = "Mediocre"
ref_cross_matrix.loc[(row_fill_value, "Excellent"), (column_fill_value, "Bad")] = "Good"
ref_cross_matrix.loc[(row_fill_value, "Excellent"), (column_fill_value, "Stable")] = "Mediocre"
ref_cross_matrix.loc[(row_fill_value, "Excellent"), (column_fill_value, "Excellent")] = "Excellent"

# Style the dataframe.
ref_cross_matrix = ref_cross_matrix.style.apply(cross_matrix_set_background_color, axis=None).pipe(make_pretty, title="Reference Cross LULC and NDVI Zone Change matrix")

In [6]:
# Load the LULC change xarray.DataArray for the initial and follow up year defined above.
lulc_change_fn = f"{output_dir}/{country.replace(' ', '')}_{initial_year}_to_{followup_year}_lulc_change.pickle"

with open(lulc_change_fn, 'rb') as handle:
    lulc_change = pickle.load(handle)

In [7]:
# Load the NDVI change xarray.DataArray for the initial and follow up year defined above.
ndvi_change_fn = f"{output_dir}/{country.replace(' ', '')}_{initial_year}_to_{followup_year}_ndvi_change.pickle"

with open(ndvi_change_fn, 'rb') as handle:
    ndvi_change = pickle.load(handle)

In [8]:
# Generate a legend for the comparison of the LULC and NDVI change xarray.DataArrays.
legend = {"Excellent": 2.0,
          "Good": 1.5,
          "Stable": 0.0,
          "Bad": 0.5,
          "Mediocre": 1.0}

In [9]:
# To compare the LULC and NDVI change arrays
# add the two arrays and divide the results by 2.
results = (lulc_change + ndvi_change) / 2

In [10]:
# Get the resolution i.e. actual ground distance represented by the length of a single pixel in meters.
pixel_length = results.geobox.resolution[1]
# Conversion from metres sqaured to hectares.
m2_per_ha = 10000
# Get the actual ground area represented by each pixel.
area_per_pixel = (pixel_length ** 2) / m2_per_ha

In [11]:
# Use the numpy np.unique function to return the pixel count for each 
# unique class value in results.
counts = np.unique(results, return_counts=True)
# Calculate the area of each class.
area = np.array(counts[1] * area_per_pixel)
# Map the unique class values to their areas. 
# Remove the last value as it represents the area of pixels with the value np.nan.
area_dict = dict(zip(counts[0][:-1], area[:-1]))
 
area_dict

{0.0: 13433094.0, 0.5: 1609650.0, 1.0: 396855.0, 1.5: 9495.0, 2.0: 729.0}

In [12]:
# Generate the Cross LULC and NDVI Zone Change Matrix.
cross_matrix = ref_cross_matrix.data.copy(deep=True)

for i in legend.keys():
    cross_matrix.replace(to_replace=i, value=area_dict[legend[i]], inplace=True) 
                         
cross_matrix = cross_matrix.style.apply(cross_matrix_set_background_color, axis=None).pipe(make_pretty, title=f"{initial_year} to {followup_year} {country} Cross LULC and NDVI Zone Change matrix (hectares)")                         

In [13]:
display(ref_cross_matrix)

In [14]:
display(cross_matrix)

In [15]:
# Export the above Cross LULC and NDVI zone change matrices as csv files.
ref_cross_matrix.data.to_csv(f"{output_dir}/{country.replace(' ', '')}_reference_cross_change_matrix.csv", index=False)
cross_matrix.data.to_csv(f"{output_dir}/{country.replace(' ', '')}_{initial_year}_to_{followup_year}_cross_change_matrix.csv", index=False)